# un_label_L1_L2_ML_predict
  - module name: un_label_L1_L2_ML_predict.ipynb
  - Purpose: Load pickle file and predict unlable data 

In [1]:
def prepare_data_1():
    pgm = inspect.currentframe().f_code.co_name  
    start_time = time.time() 
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object) # (5082212, 24)
    
    print(f'Before drop pim_gtin_mapped.shape {pim_gtin_mapped.shape}')    
    SUBCOMS_excluded = ['INVENTORY VALUES','MISC SALES TRANS','MISC SALES TRANS (NON TAX)','CENTRAL SUPPLIES', \
                    'MISC SALES TRANS (TX TAXABLE)','COUPON','MISCELLANEOUS INCOME','CRV/EXCISE TAX NT/NF', \
                    'CRV DEPOSIT NT/F','CRV DEPOSIT T/NF','MISCELLANEOUS REFUNDS','CUSTOMER EXPENSES','ROUND UP COUPONS']
    pim_gtin_mapped = pim_gtin_mapped[~pim_gtin_mapped.SUBCOM_DSC.isin(SUBCOMS_excluded)]   
    print(f'After drop pim_gtin_mapped.shape {pim_gtin_mapped.shape}')
    
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
         pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # drop 'Level 1' ~ 'Level 10'
    l_rmv_col =['ITM_ID_y','Level 1', 'Level 2',
       'Level 3', 'Level 4', 'Level 5', 'Level 6', 'Level 7', 'Level 8',
       'Level 9', 'Level 10']
    pim_gtin_mapped.drop(columns = l_rmv_col, inplace = True ) 

    pim_gtin_mapped.drop_duplicates('GTIN', inplace = True)
    pim_gtin_mapped['ITEM_SUBCOM_text'] = \
    (pim_gtin_mapped.VND_ECOM_DSC + ' ' + pim_gtin_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    #print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    # Memory issue; Divid pim_gtin_mapped into two df for predecting
    pim_gtin1, pim_gtin2, pim_gtin3, pim_gtin4 =  np.array_split(pim_gtin_mapped, 4)  
    return pim_gtin_mapped, pim_gtin1, pim_gtin2, pim_gtin3, pim_gtin4

In [2]:
def prepare_data_2(pim_gtin, embed_typ = 'tfidf'):
  
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    global model
  
    if embed_typ == 'tfidf': 
        vect = TfidfVectorizer(ngram_range = (1,2), max_features = 50000) 
        X = vect.fit_transform(pim_gtin.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    else:
        model = KeyedVectors.load_word2vec_format(DBFO + 'w2vmodel_053123_PIM_ALL.bin', binary=True)
        X = np.array(list(pim_gtin.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))

    
    end_time = time.time()   
    desc = f'Process DF {df} with elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return  X

In [3]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [5]:
def proc_pred(embed_typ):
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    save_model =  DBFR + f'L1_l2_LR_{embed_typ}_tst_siz0.3_C100_083123.pkl'
    print (f' proc_pred for embed_typ = {embed_typ}')
    with open(save_model , 'rb') as f:
         model = pickle.load(f)
    preds = list(model.predict(X))    
    probx =model.predict_proba(X)
    probs = []
    for i in range(len(preds)):
      probs.append(probx[i][np.argsort(probx[i])][::-1][:1])
    probs = [element for sublist in list(probs) for element in sublist]
    
    data = {'Predict': preds, 'Scores': probs}
    df_pred= pd.DataFrame(data)
    df_pred['Predict'] = df_pred['Predict'].fillna('Other').map(id_l1_l2_map) 
    df_merg = pd.merge( pim_gtin_mapped, df_pred, left_index=True, right_index=True)

    end_time = time.time() 
    desc = f' Elapse_time to predict un-label data for embed_typ= {embed_typ}' 
    hh, mm, ss= elapse_time (  start_time, end_time, desc)
                                   
    return df_merg

In [6]:
def proc_save(df,embed_typ):
   dte = date.today().strftime('%m%d%y')  
   file_name = DBFR + f'l1_l2_unlabel_pred_map_{embed_typ}_{dte}.csv'
   df.to_csv(file_name, index=None)
   return

In [7]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from datetime import date


from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
import re, time, inspect, pickle
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pd.options.display.float_format = '{:20,.4f}'.format 
pd.set_option("display.max_rows", None)

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))


In [8]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [9]:
pim_gtin_mapped, pim_gtin1, pim_gtin2,pim_gtin3, pim_gtin4 = prepare_data_1()
pim_gtin_mapped_bkup =  pim_gtin_mapped.copy()
pim_gtin1_bkup =  pim_gtin1.copy()
pim_gtin2_bkup =  pim_gtin2.copy()
pim_gtin3_bkup =  pim_gtin3.copy()
pim_gtin4_bkup =  pim_gtin4.copy()


Before drop pim_gtin_mapped.shape (5082212, 24)
After drop pim_gtin_mapped.shape (5061471, 24)


NameError: name 'pim_gtin4copy' is not defined

In [10]:
pim_gtin4_bkup =  pim_gtin4.copy()

In [13]:
 
l_embed_typ = ['tfidf', 'W2vec']
l_embed_typ = ['W2vec']
c_parm = 100
l_df= [pim_gtin1, pim_gtin2, pim_gtin3, pim_gtin4 ]


#l_embed_typ = ['tfidf', 'W2vec']

map_csv = pd.read_csv(DBFR + "id_l1_l2_map.csv")
l_key = map_csv['index']
l_val = map_csv['l1_l2']
id_l1_l2_map =  dict (zip(l_key, l_val))

samp_frac = 0.5

for embed_typ in l_embed_typ:
    i = 1
    for df in l_df: 
        print (f' process pim _gtin{i} for {embed_typ}' )
        X= prepare_data_2(df, embed_typ = embed_typ )
        globals()[f'df{i}'] = proc_pred(embed_typ)
        i = i + 1 
    globals()[f'df_{embed_typ}_cont'] =  pd.concat([df1, df2, df3, df4])
    proc_save(globals()[f'df_{embed_typ}_cont'], embed_typ)



 process pim _gtin1 for W2vec


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  Elapse_time to predict un-label data for embed_typ= W2vec It took 36.851799 seconds - 0hh:0mm:36ss.
 start time: Aug 31 2023 21:16:42  end time:  Aug 31 2023 21:17:19
 process pim _gtin2 for W2vec


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  Elapse_time to predict un-label data for embed_typ= W2vec It took 31.331971 seconds - 0hh:0mm:31ss.
 start time: Aug 31 2023 21:19:27  end time:  Aug 31 2023 21:19:59
 process pim _gtin3 for W2vec


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  Elapse_time to predict un-label data for embed_typ= W2vec It took 34.802090 seconds - 0hh:0mm:34ss.
 start time: Aug 31 2023 21:22:14  end time:  Aug 31 2023 21:22:49
 process pim _gtin4 for W2vec


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  Elapse_time to predict un-label data for embed_typ= W2vec It took 29.999414 seconds - 0hh:0mm:29ss.
 start time: Aug 31 2023 21:25:06  end time:  Aug 31 2023 21:25:36


In [ ]:
stop

In [14]:
df_W2vec_cont.columns

Index(['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
       'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
       'SUBCOM_DSC', 'VND_ECOM_DSC', 'ITEM_SUBCOM_text', 'Predict',
       'Accuracy'],
      dtype='object')

In [15]:
df_W2vec_cont.shape

(5012719, 16)

In [16]:
df_W2vec_cont.rename(columns = {'Accuracy: Scores'}, inplace = True)

TypeError: 'set' object is not callable

In [ ]:
df_W2vec_cont.head()

In [ ]:
df_W2vec_cont = df_W2vec_cont.rename(columns= {'Accuracy: Probality'})

rankings_pd.rename(columns = {'test':'TEST', 'odi':'ODI',
                              't20':'T20'}, inplace = True)

In [ ]:
save_model =  DBFR + 'L1_l2_LR_tfidf_tst_siz0.3_C100.pkl'
with open(save_model , 'rb') as f:
     model = pickle.load(f)

In [ ]:
preds = model.predict(X)  # max(pred)+  200; type[array([ 1, 20, 12, ...,  6, 20,  6], dtype=int64)]

In [ ]:
pred.fillna('Other').map(id_l1_l2_map)

In [ ]:
predx = list(preds)
probx = list(probs)
type(predx)

In [ ]:
data = {'preds': predx, 'probs': probx}
df_pred= pd.DataFrame(data)
df_pred

In [ ]:
probs = model.predict_proba(X)  #numpy.ndarray; len = 1524664

In [ ]:
len(pred)

In [ ]:
path = 'C:\\users\\iny2819\\kroger\\Data\\'
map_csv = pd.read_csv(path + "id_l1_l2_map.csv")
l_val = map_csv['index']
l_key = map_csv['l1_l2'].tolist()
id_l1_l2_map =  dict (zip(l_key, l_val))

In [ ]:
map_csv.head()

In [ ]:
l_c_parm = a0.1, 1, 10, 100] 
l_embed_typ = ['tfidf', 'W2vec']

dic_metric={}
dic_predict = {}
l_metric=[]
test_size = 0.3
i = 0
for embed_typ in l_embed_typ:
    X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 1, embed_typ = embed_typ )
    for c_parm in l_c_parm:
        start_time = time.time()        
        
        preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3, c_parm =c_parm,  embed_typ= embed_typ )
 
        test_metrics, vtab, l_metrics  =  validation(test_size = 0.3, c_parm =c_parm, embed_typ=embed_typ) 
        key = 'test_metric' + str(i)
        dic_metric [key] = (test_metrics, vtab)
        
        test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss = proc_append()
        df_save, ftab  = proc_cr_df(test_size=0.3, c_parm=c_parm, embed_typ= embed_typ )
        key = 'predict'+ str(i)
        dic_predict[key] = (df_save, ftab)
        i = i + 1   
        end_time = time.time() 
        desc = f' Elapse_time to build model for for test_size = 0.3, c_parm = {c_parm}, embed_typ= {embed_typ}' 
        hh, mm, ss= elapse_time (  start_time, end_time, desc)
                                 
        l_metrics = l_metrics +  [f"{hh:2d}hh:{mm:2d}mm:{ss:2d}ss"]   
        l_metric.append(l_metrics)
dte = date.today().strftime('%m%d%y')                                  
excel_file = DBFO + f'L1_L2_syndigo_pred_cls_rpt_size{test_size}_{dte}.xlsx'
save_excel_metric( excel_file, dic_metric, l_metric)


excel_file = DBFO + f'L1_L2_syndigo_pred_tst_size{test_size}_{dte}.xlsx'
save_excel_pred( excel_file, dic_predict)

In [ ]:
stop

In [ ]:
excel_file = DBFO + f'L1_L2_syndigo_pred_tst_size{test_size}_082527.xlsx'
save_excel( excel_file, dic_predict)

In [ ]:
dic_predict.items()

In [ ]:
test_size = 0.3 
excel_file = DBFO + f'L1_L2_syndigo_pred_cls_rpt_size{test_size}_082527.xlsx'
save_excel_metric( excel_file, dic_metric, l_metric)


excel_file = DBFO + f'L1_L2_syndigo_pred_tst_size{test_size}_082527.xlsx'
save_excel_pred( excel_file, dic_predict)

In [ ]:
multi_class = 'multinomial'
score = roc_auc_score(y_test, preds, multi_class= multi_class)
print(f"ROC AUC score for Multiclass= {multi_class}: {score:.2f}")

In [ ]:
pim_gtin_mapped.columns

In [ ]:
pim_gtin_mapped.shape

In [ ]:
pim_gtin_mapped.head()